In [1]:
import pandas as pd
import numpy as np
import csv

# library for storing and loading objects
import pickle

# library for stemming work
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# library used to make inverted index
from collections import defaultdict
from collections import Counter

# cosine distance
from scipy.spatial.distance import cosine

import math 

# heap_max library
import heapq
from heapq import heappush, heappop

from geopy.geocoders import Nominatim
from geopy import distance
import functions

# variable declarations
path1 = 'data/docs/'
path2 = '.tsv'
#stemmed_path='data/tokenized_docs/'
#sp = string.punctuation+'“”–’'

[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 2: Create documents
    - Importing data from Airbnb_Texas_Rentals.csv and preparing it for pre processing

In [6]:
texas1 = pd.read_csv("data/Airbnb_Texas_Rentals.csv")
texas1 = texas1.drop(['Unnamed: 0'], axis=1)
texas1 = texas1.drop_duplicates()
texas1 = texas1[np.isfinite(texas1['latitude'])]
texas1 = texas1[np.isfinite(texas1['longitude'])]


18225


,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
18254,$60,1,Dallas,March 2013,An entire 1 bedroom 700+sqft condo in the hear...,32.892303,-96.772049,Quiet comfort living in Dallas,https://www.airbnb.com/rooms/1011576?location=...
18255,$99,2,San Antonio,June 2015,An inviting 1920's cottage home in a popular u...,29.452893,-98.486756,Midtown Cottage Near Riverwalk,https://www.airbnb.com/rooms/18766940?location...
18256,$13,1,Dallas,December 2016,Amazing 3BHK Apartment in a picturesque Commun...,33.001955,-96.777615,Room in Dallas!,https://www.airbnb.com/rooms/18719059?location...
18257,$65,2,San Antonio,October 2016,My quaint and cozy home is conveniently locate...,29.450142,-98.505333,Vibrant Spacious Loft!,https://www.airbnb.com/rooms/18179329?location...
18258,$75,1,San Antonio,July 2015,"Cute brand new, modern, guest house (we call i...",29.425508,-98.466833,Brand New Modern Pod in Historic Dignowity Hill,https://www.airbnb.com/rooms/16318144?location...


Preprocessing create the tsv files.

We created vocabulary parsing every document and updating it when the algorithm finds a new word. 

Format of vocabulary => 'string':integer.

Creation of ii1:

The first inverted index is built, updating it every time a word is find in a document.
At the end we stored vocabulary and the inverted index using pickle.

In [26]:
from importlib import reload
reload(functions)

[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'functions' from '/Users/miguel/Desktop/HW3/ADM18-HW3-Group29/functions.py'>

# Step 3: Search Engine

### 3.1.1) Preprocessing

Store rows of dataframe as separate tsv files into a folder

In [12]:
functions.preprocessing(texas1)

### Creating the vocabulary and the inverted index 1
We decided to create the vocabulary and inverted index 1 into the same methods because it's faster... 


#### WE HAVE TO OCOMMENT

In [13]:
functions.create_vocabulary_and_ii1(texas1)

## 3.1) Search engine 1

In [127]:
reload(functions)

[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'functions' from '/Users/miguel/Desktop/HW3/ADM18-HW3-Group29/functions.py'>

In [128]:
res = functions.search_engine_1('House with garden')
res

,Title,Description,City,Url
1,Zen & the Art of the Ranch House,A much-loved family home nestled in a serene g...,Irving,https://www.airbnb.com/rooms/11502964?location...
2,The Fortress for your comfort.,The Fortress is a two story ranch style house ...,Austin,https://www.airbnb.com/rooms/11160635?location...
3,GraceGardens: Rustic Oak Cabin,Enchanting retreat on the San Marcos River in ...,Martindale,https://www.airbnb.com/rooms/964091?location=B...
4,"Rosen House Inn, Rose Garden Room",We are a small family owned Bed and Breakfast ...,Fort Worth,https://www.airbnb.com/rooms/9972447?location=...
5,Arlington Heights Guest House Cottage,"Bright,Open and Airy Cottage. Great location t...",Fort Worth,https://www.airbnb.com/rooms/18557788?location...


## 3.2) Conjunctive query & Ranking score

### 3.2.1) Inverted index

In [5]:
functions.inverted_index_TFIDF(texas1)

### 3.2.2) Execute the query

We decided to create the new search engine (second) basing on the inverted indexes already built, but cutting them: in this way the laptop will have less computation than to compute everything.
Thus, every time we have a new query, the method generates 2 different inverted indexes.

In this way, we won't need a big numpy array for the query and each document considered (we took docs that contained every word, using intersection (for this purpose we took the first inverted index)) for making the cosine similarity (for this we exploited the second inverted index) ; in fact, vectors for computing cosine similarity will be exactly long like the length of the query. 
Query will have 1 for each element (?)(still to decide) and doc vector will have each tfidf for each term of the query.

$$ 1-cosine(\text{doc array for the query},\text{array of query}) $$
will compute the Cosine Similarity



In [129]:
res = functions.search_engine_2('Beach, two rooms with private bathroom')
res

,Title,Description,City,Url,Similarity
1,Allegro North First Floor Condo for 7 guests,"Two bedroom, two bathroom condo with master su...",Rockport,https://www.airbnb.com/rooms/18157657?location...,0.864
2,"Condo 305 is Comfortable, Economical on beach",Our home is designed for comfort and convenien...,South Padre Island,https://www.airbnb.com/rooms/11580397?location...,0.855
3,North Padre Island room beach 1M,My wife and I will welcome you into our home a...,Corpus Christi,https://www.airbnb.com/rooms/3767705?location=...,0.855
4,North Padre Island room beach 1M,My wife and I will welcome you into our home a...,Corpus Christi,https://www.airbnb.com/rooms/3767705?location=...,0.855
5,Private Bedroom (queen)/Bathroom in Beautiful ...,Beautiful 3 bedroom/2 bath house (1 private gu...,Corpus Christi,https://www.airbnb.com/rooms/19484822?location...,0.708


# Step 4: Define a new score!

### 'search_engine_1' is the function which assigns ranking for each documents from some information inputted by users. 

As we have done in the previous question 3.1 with searching engine 1, this new method will bring some documents which satisfy a text query at first. 
Moreover, besides the a text query, it allows the users to input 'target price (price)', 'the number of rooms (room)' and 'location'. These are the attributes whith whom we will evaluate documents matched with search engine 1 algorithm.

Each rank takes values from 0 to 1.
In this way we are able to make a weighted average, giving different weight to the attributes.
We weigh target price, the number of rooms and location as 0.3 , 0.1 and 0.6 respectively. 
It is obvious that the location should have more weight among these information. In addition, we make a difference between target price and the number of rooms, since we consider the price would be more important than the number of rooms.

### In terms of scoring by each information, we create functions to give the points to each different document. 

#### Target price : ( Function name is 'price_rank' )
We score the low point if the difference between the target price that users desire and price of documents is big. We decide to make the score as a linear function that decrease linearly with the absolute difference between target price and document price with slope of 0.01. In this case we are able to spread ranks related to the target price in a very smooth way. Since we want a rank value between 0 and 1, we had to define properly the domain of our function. 

$$ \text{price_rank}_\text{doc_i} = 1 - \frac{1}{100} \times \left| \text{ price_target} - \text{price_doc_i } \right| $$

In case that the price difference is equal or greater than 100, the price rank is 0 point.

#### The number of rooms:  ( Function name is 'rooms_rank' )
As with target price, we assign each point according to the difference between the number of rooms that users desire and that of documents. 

When the difference is 0, 1, 2 and 3, it returns 1, 0.7, 0.4 and 0.1 respectively. We regard studio room as single room. 
If it becomes more than 4, it returns 0. 

#### Location : ( Function name is 'distance_rank' ) 
It gives points depending on the distance from the location desired to the bnb. 
We use geopy to find the latitude and longitude of locations and to calculate the distance between the place searched and the position of the documents.ude of locations and to calculate the distance between 2 places.  

Once we find this out, the funciton that returns points is as follows. 
$$  \text{location_rank}_\text{doc_i}  = 1 -  \frac{\text{distance}}{1 + \text{distance}}$$
It is clear that the farther the distance is, the lower the point is. Therefore, we apply this function because it goes smoothly to 0 from 1 asymptotically, in this way, after we compute the distance, we can take the value 1 - function, to get results inversely proportional to the radius. 

In [29]:
functions.search_engine_3('Swimming pool')

target price:
80
n rooms:
2
location
Dallas
Found 409 documents


,Ranking,Title,Description,City,Url
0,1,G's Resort V 2.2,Convenient central location near SMU with priv...,Dallas,https://www.airbnb.com/rooms/8341908?location=...
1,2,G's Resort V 2.2,Convenient central location near SMU with priv...,Dallas,https://www.airbnb.com/rooms/8341908?location=...
2,3,Mi casa es su Casa. Travel instyle.,"Gorgeous 3 master suites w/ privatel baths, al...",Dallas,https://www.airbnb.com/rooms/790791?location=A...
3,4,The Suite,"'The Suite' has a large bedroom, 2 bathrooms, ...",Arlington,https://www.airbnb.com/rooms/2617711?location=...
4,5,The Suite,"'The Suite' has a large bedroom, 2 bathrooms, ...",Arlington,https://www.airbnb.com/rooms/2617711?location=...


### Step 4: make nice visualization

In [3]:
import folium

In [125]:
location = 'Dallas'
km = 6

geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode(location)
print(location)
center = (location.latitude, location.longitude)

m = folium.Map(center, zoom_start=10)
folium.Marker(location = center).add_to(m)
#folium.Marker(center,mark).add_to(m)
folium.Circle(center, radius= km * 1000, color = 'blue', fill = True).add_to(m)
#m
#no_wrap = True restricts panning while max_bounds = True

Dallas, Dallas County, Texas, USA


In [102]:
docs = ['doc_2']

for doc in docs:
    with open('data/docs/'+doc+'.tsv', encoding = 'utf-8') as f:
        row = f.read()
        row = row.split('\t')
        nrooms = row[1]
        
        house = (float(row[5]),float(row[6]))
        
        
        if distance.distance(center, house).km <= km:
            folium.Marker(location = house, icon=folium.Icon(color='green')).add_to(m)
            
#m

In [127]:
location = 'Dallas'
km = 6

geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode(location)
print(location)
center = (location.latitude, location.longitude)
m = folium.Map(center, zoom_start=12)
folium.Marker(location = center).add_to(m)
#folium.Marker(center,mark).add_to(m)
folium.Circle(center, radius= km * 1000, color = 'blue', fill = True).add_to(m)

for i in range(18259):
    with open('data/docs/doc_'+str(i)+'.tsv', encoding = 'utf-8') as f:
        row = f.read()
        row = row.split('\t')
        nrooms = row[1]
        
        house = (float(row[5]),float(row[6]))
        
        riga = row[7] + '\n' +row[8]
        if distance.distance(center, house).km <= km:
            folium.Marker(location = house, 
                          icon=folium.Icon(color='green'),
                          popup = row[8]).add_to(m)
            
m

Dallas, Dallas County, Texas, USA
